<a href="https://colab.research.google.com/github/ebuehrle/william/blob/main/gpt2_conv_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/huggingface/transfer-learning-conv-ai

Cloning into 'transfer-learning-conv-ai'...
remote: Enumerating objects: 132, done.
remote: Total 132 (delta 0), reused 0 (delta 0), pack-reused 132
Receiving objects: 100% (132/132), 56.09 KiB | 2.55 MiB/s, done.
Resolving deltas: 100% (71/71), done.


In [3]:
%cd transfer-learning-conv-ai/

/content/transfer-learning-conv-ai


In [4]:
!pip install -r requirements.txt -q
!python -m spacy download en

     |████████████████████████████████| 225kB 4.1MB/s 
     |████████████████████████████████| 501kB 13.7MB/s 
     |████████████████████████████████| 225kB 21.2MB/s 
     |████████████████████████████████| 133kB 20.2MB/s 
     |████████████████████████████████| 1.2MB 19.4MB/s 
     |████████████████████████████████| 901kB 33.2MB/s 
     |████████████████████████████████| 5.6MB 40.9MB/s 
     |████████████████████████████████| 7.7MB 42.4MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
ERROR: botocore 1.20.101 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [5]:
#!python interact.py

In [6]:
import argparse
import logging
from pprint import pformat
import torch
import random
import itertools

In [7]:
from transformers import OpenAIGPTLMHeadModel, OpenAIGPTTokenizer, GPT2LMHeadModel, GPT2Tokenizer
from train import SPECIAL_TOKENS, build_input_from_segments, add_special_tokens_
from utils import get_dataset, download_pretrained_model
import interact

In [8]:
def tokenize(obj, tokenizer):
  if isinstance(obj, str):
      return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(obj))
  if isinstance(obj, dict):
      return dict((n, tokenize(o, tokenizer)) for n, o in obj.items())
  return list(tokenize(o, tokenizer) for o in obj)

In [9]:
def setup_chatbot(personality=None, **kwargs):
  args = argparse.Namespace(
    dataset_path = kwargs.get("dataset_path", ""), # Path or url of the dataset. If empty download from S3.
    dataset_cache = kwargs.get("dataset_cache", "./dataset_cache"), # Path or url of the dataset cache
    model = kwargs.get("model", "openai-gpt"), # Model type (openai-gpt or gpt2). Anything besides gpt2 will load openai-gpt.
    model_checkpoint = kwargs.get("model_checkpoint", ""), # Path, url or short name of the model
    max_history = kwargs.get("max_history", 2), # Number of previous utterances to keep in history
    device = kwargs.get("device", "cuda" if torch.cuda.is_available() else "cpu"), # Device (cuda or cpu)
    no_sample = kwargs.get("no_sample", False), # Set to use greedy decoding instead of sampling
    max_length = kwargs.get("max_length", 20), # Maximum length of the output utterances
    min_length = kwargs.get("min_length", 1), # Minimum length of the output utterances
    seed = kwargs.get("seed", 0), # Seed
    temperature = kwargs.get("temperature", 0.7), # Sampling softmax temperature
    top_k = kwargs.get("top_k", 0), # Filter top-k tokens before sampling (<=0: no filtering)
    top_p = kwargs.get("top_p", 0.9), # Nucleus filtering (top-p) before sampling (<=0.0: no filtering)
  )

  logging.basicConfig(level=logging.INFO)
  logger = logging.getLogger()
  logger.info(pformat(args))

  if args.model_checkpoint == "":
      if args.model == 'gpt2':
          raise ValueError("Interacting with GPT2 requires passing a finetuned model_checkpoint")
      else:
          args.model_checkpoint = download_pretrained_model()

  if args.seed != 0:
    random.seed(args.seed)
    torch.random.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)

  logger.info("Get pretrained model and tokenizer")
  tokenizer_class, model_class = (GPT2Tokenizer, GPT2LMHeadModel) if args.model == 'gpt2' else (OpenAIGPTTokenizer, OpenAIGPTLMHeadModel)
  tokenizer = tokenizer_class.from_pretrained(args.model_checkpoint)
  model = model_class.from_pretrained(args.model_checkpoint)
  model.to(args.device)
  add_special_tokens_(model, tokenizer)

  if personality:
    personality = tokenize(personality, tokenizer)
  else:
    logger.info("Sample a personality")
    dataset = get_dataset(tokenizer, args.dataset_path, args.dataset_cache)
    personalities = [dialog["personality"] for dataset in dataset.values() for dialog in dataset]
    personality = random.choice(personalities)

  logger.info("Selected personality: %s", tokenizer.decode(itertools.chain(*personality)))
  
  def _respond(history):
    history = [tokenizer.encode(h) for h in history]
    with torch.no_grad():
        out_ids = interact.sample_sequence(personality, history, tokenizer, model, args)
    response = tokenizer.decode(out_ids, skip_special_tokens=True)
    return response
  
  return _respond

In [18]:
persona = ["my name is charles william .",
           "i am from germany .",
           "i was born in 1679 .",
           "my brother died in 1672 .",
           "i founded the city of karlsruhe .",
           "i was a politician ."]

In [19]:
respond = setup_chatbot(persona)

INFO:root:Namespace(dataset_cache='./dataset_cache', dataset_path='', device='cpu', max_history=2, max_length=20, min_length=1, model='openai-gpt', model_checkpoint='', no_sample=False, seed=0, temperature=0.7, top_k=0, top_p=0.9)
INFO:/content/transfer-learning-conv-ai/utils.py:extracting archive file /root/.cache/torch/transformers/2f5114b5eb72f9515802779c42c1b289bebdb1cfc8ce94c653237518eb530b34.75f2a4fe69178ff43138117a977e107a5fc7d402603a0825a296b531f246b3f2 to temp dir /tmp/tmpcn_1asa_
INFO:root:Get pretrained model and tokenizer
INFO:transformers.tokenization_utils:Model name '/tmp/tmpcn_1asa_' not found in model shortcut name list (openai-gpt). Assuming '/tmp/tmpcn_1asa_' is a path, a model identifier, or url to a directory containing tokenizer files.
INFO:transformers.tokenization_utils:Didn't find file /tmp/tmpcn_1asa_/special_tokens_map.json. We won't load it.
INFO:transformers.tokenization_utils:Didn't find file /tmp/tmpcn_1asa_/tokenizer_config.json. We won't load it.
INFO:t

In [20]:
history = ["hello how are you?",
           "i am fine thanks.",
           "where are you from"]

In [21]:
response = respond(history)
print(response)

germany.


# REST Endpoint

In [14]:
!pip install flask-ngrok
!pip install flask-cors
!pip install flask==0.12.2  # Newer versions of flask don't work in Colab
                            # See https://github.com/plotly/dash/issues/257

     |████████████████████████████████| 92kB 3.2MB/s 
  Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4


In [22]:
from flask import Flask, Response
from flask_ngrok import run_with_ngrok
from flask import request
from flask_cors import CORS

app = Flask(__name__)
CORS(app)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/", methods=['POST'])
def hello():
  history = request.get_json()
  print(history)
  response = respond(history)
  #response = "dummy response"
  return response

@app.route("/ping")
def ping():
  return "hello world"

if __name__ == '__main__':
    app.run()  # If address is in use, may need to terminate other sessions:
               # Runtime > Manage Sessions > Terminate Other Sessions

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://819c1b1a2f99.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [26/Jun/2021 13:52:47] "OPTIONS / HTTP/1.1" 200 -


['who are you']


...

INFO:werkzeug:127.0.0.1 - - [26/Jun/2021 14:14:15] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jun/2021 14:20:31] "OPTIONS / HTTP/1.1" 200 -


['how are you', "i'm doing well. how are you?", 'what is on tv tonight']


INFO:werkzeug:127.0.0.1 - - [26/Jun/2021 14:20:37] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jun/2021 14:20:53] "OPTIONS / HTTP/1.1" 200 -


['how are you', "i'm doing well. how are you?", 'what is on tv tonight', 'it is a good show. i hope you like it.', 'i am not sure if I will like it']


INFO:werkzeug:127.0.0.1 - - [26/Jun/2021 14:21:01] "POST / HTTP/1.1" 200 -
